In [ ]:
import cv2
import numpy as np
import tensorflow as tf

model = tf.keras.models.load_model('models/mnist.keras')

drawing  = False

prev_x   =  -1
prev_y   =  -1   

img = np.zeros((500,500,3),np.uint8)
text_bg = np.full((60,500,3),(255,255,255),np.uint8)
cv2.putText(text_bg,"Can We Guess The Number?",(20,35),cv2.FONT_ITALIC,1,(0,0,255),2)
img[0:60, :] = text_bg

def call_back(event,x,y,flags,params):

    global drawing,img,prev_x,prev_y,brush_size

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing  = True
        prev_x,prev_y = x,y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            brush_size = cv2.getTrackbarPos("Brush Size","Test")
            if brush_size < 1:
                brush_size = 1
            cv2.line(img,(prev_x,prev_y),(x,y),(255,255,255),brush_size)
            prev_x = x
            prev_y = y

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        input_img = img[60:, :, 0] 
        input_img = cv2.resize(input_img, (28, 28))  
        input_img = input_img / 255.0
        input_img = input_img.reshape(1, 28, 28, 1)
        pred = model.predict(input_img)
        result = np.argmax(pred)

        text_bg = np.full((60,500,3),(255,255,255),np.uint8)
        cv2.putText(text_bg,f"Is The Number {result}?",(20,35),cv2.FONT_ITALIC,1,(0,0,255),2)
        img[0:60, :] = text_bg
    
    elif event == cv2.EVENT_RBUTTONDOWN:
        img = np.zeros((500,500,3),np.uint8)
        text_bg = np.full((60,500,3),(255,255,255),np.uint8)
        cv2.putText(text_bg,"Can We Guess The Number?",(20,35),cv2.FONT_ITALIC,1,(0,0,255),2)
        img[0:60, :] = text_bg

        
cv2.namedWindow("Test")
cv2.setMouseCallback("Test",call_back)

def nothing(x):
    pass

cv2.createTrackbar("Brush Size","Test",1,100,nothing)

while True:
    cv2.imshow("Test",img)

    k=cv2.waitKey(1)

    if k==ord("q"):
        break
cv2.destroyAllWindows()